# Similarity Score

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem


class ZAtomType:
    """获取特征"""
    
    def __init__(self, mol):
        self.mol = mol

    def read_sdf(file_path):
        # Readtwo SDF files with rdkit
        self.mol = Chem.SDMolSupplier(file_path)[0]

    def get_atom_ring_types(self):
        # 获取每个原子的环类型，包括8种类型：
        # {0: 非环, 1: 3元环, 2: 4元环, 3: 5元环, 4: 6元环, 5: 5&6元环, 6: 6&6元环, 7: 其他环}
        ring_info = self.mol.GetRingInfo()
        atom_rings = ring_info.AtomRings()

        # get all atoms in ring
        dict_atom_ringsize = dict()
        for r in atom_rings:
            for i in r:
                if i in dict_atom_ringsize:
                    dict_atom_ringsize[i].append(len(r))
                else:
                    dict_atom_ringsize[i] = [len(r)]

        atom_ring_types = []
        # 获取所有环，按原子索引
        for i in range(self.mol.GetNumAtoms()):
            at = 7

            if i not in dict_atom_ringsize:
                at = 0
            
            elif len(dict_atom_ringsize[i]) == 1:
                if ring_info.IsAtomInRingOfSize(i, 3):
                    at = 1
                elif ring_info.IsAtomInRingOfSize(i, 4):
                    at = 2
                elif ring_info.IsAtomInRingOfSize(i, 5):
                    at = 3
                elif ring_info.IsAtomInRingOfSize(i, 6):
                    at = 4
            
            elif len(dict_atom_ringsize[i]) == 2:
                if set(dict_atom_ringsize[i]) == {5, 6}:
                    at = 5
                elif set(dict_atom_ringsize[i]) == {6}:
                    at = 6

        
            atom_ring_types.append(at)

        return atom_ring_types


    def get_atom_charge_types(self, ff=False):
        # 补氢
        self.mol = Chem.AddHs(self.mol)  
        
        atom_charges = []

        if ff: # MMFF94
            # 生成初始构象
            AllChem.EmbedMolecule(self.mol)  

            # 优化分子构象
            AllChem.MMFFOptimizeMolecule(self.mol)

            # 计算 MMFF94 电荷
            mp = AllChem.MMFFGetMoleculeProperties(self.mol)
            for atom in self.mol.GetAtoms():
                charge = mp.GetMMFFPartialCharge(atom.GetIdx())
                atom_charges.append(charge)

        else: # Gasteiger 电荷
            AllChem.ComputeGasteigerCharges(self.mol)

            for atom in self.mol.GetAtoms():
                charge = atom.GetProp('_GasteigerCharge')
                atom_charges.append(charge)

        return atom_charges


    def get_atom_bond_number(self):
        atom_bond_number = [len(atom.GetBonds()) for atom in self.mol.GetAtoms()]
        return atom_bond_number


    def get_atom_element_types(self):
        atom_element_types = [atom.GetSymbol() for atom in self.mol.GetAtoms()]
        return atom_element_types
    



In [ ]:
import numpy as np

class ZScore:
    def __init__(self, mol_q, mol_t):
        self.at_q = ZAtomType(mol_q)
        self.at_t = ZAtomType(mol_t)

    def score_element(self):
        # 1 if at_q == at_t, 0 otherwise
        # size=m list and size=n list to create a m*n matrix
        # 扩展两个矩阵，生成 m*n 的比较矩阵
        q_types = np.array(self.at_q.get_atom_element_types())
        t_types = np.array(self.at_t.get_atom_element_types())

        return (q_types[:, np.newaxis] == t_types[np.newaxis, :])

    def score_charge(self):
        """其中电荷项打分
        $$a_{\text{charge}} =\left\{
                \begin{array}{lr}
                0.25 \times (y+2)(y - 1)^2,          &-1 < y < 1 \\
                0,  &y \ge 1
                \end{array}
            \right.
        $$
        """
        q_charges = np.array(self.at_q.get_atom_charge_types())
        t_charges = np.array(self.at_t.get_atom_charge_types())

        return 0.25 * (q_charges[:, np.newaxis] + 2) * (q_charges[:, np.newaxis] - 1) ** 2 + \
        (t_charges[np.newaxis, :] + 2) * (t_charges[np.newaxis, :] - 1) ** 2

    def score_bond(self):
        q_bonds = np.array(self.at_q.get_atom_bond_number())
        t_bonds = np.array(self.at_t.get_atom_bond_number())

        return (q_bonds[:, np.newaxis] == t_bonds[np.newaxis, :])

    def score_ring(self):
        q_rings = np.array(self.at_q.get_atom_ring_types())
        t_rings = np.array(self.at_t.get_atom_ring_types())

        return (q_rings[:, np.newaxis] == t_rings[np.newaxis, :])

    def score(self, mol_q, mol_t):
        """
        $$
        a_{qt} = a_{\text{type}} + a_{\text{charge}} + \min(2, a_{\text{bond}} + 2\times a_{\text{ring}})
        $$
        """
        a_element = self.score_element()
        a_charge = self.score_charge()
        a_bond = self.score_bond()
        a_ring = self.score_ring()

        return a_element + a_charge + min(2, a_bond + 2 * a_ring)



<>:18: SyntaxWarning: invalid escape sequence '\l'
<>:46: SyntaxWarning: invalid escape sequence '\m'
<>:18: SyntaxWarning: invalid escape sequence '\l'
<>:46: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_2355225/1614240382.py:18: SyntaxWarning: invalid escape sequence '\l'
  $$a_{\text{charge}} =\left\{
/tmp/ipykernel_2355225/1614240382.py:46: SyntaxWarning: invalid escape sequence '\m'
  a_{qt} = a_{\text{type}} + a_{\text{charge}} + \min(2, a_{\text{bond}} + 2\times a_{\text{ring}})


SyntaxError: incomplete input (1614240382.py, line 59)

# Utils

## Transfer .MOL file to .SDF File

In [22]:
def transfer_mol_to_sdf(mol_file_path):
    fp_sdf = mol_file_path.replace(".mol", ".sdf")
    mol = Chem.MolFromMolFile(mol_file_path)
    Chem.MolToMolFile(mol, fp_sdf)
    return fp_sdf

transfer_mol_to_sdf("/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.mol")

'/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.sdf'

In [ ]:
$$a_{\text{charge}} =\left\{
		\begin{array}{lr}
		0.25 \times (y+2)(y - 1)^2,          &-1 < y < 1 \\
		0,  &y \ge 1
		\end{array}
	\right.
$$

# Attractive Energy

# Repulsive Energy